In [ ]:
#Import libraries

import cv2
import numpy as np
from numpy import expand_dims
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import clear_output

class ExtractImage(object):
    def __init__(self,image_name):
        self.image_name = image_name
        self.original_image = cv2.imread(self.image_name)

        self.original_image = cv2.resize(self.original_image, (600,600))
        self.clone = self.original_image.copy()

        cv2.namedWindow('image')
        cv2.setMouseCallback('image', self.extract_coordinates)
        self.image_coordinates = []

    def extract_coordinates(self, event, x, y, flags, parameters):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.image_coordinates = [(x,y)]

        elif event == cv2.EVENT_LBUTTONUP:
            self.image_coordinates.append((x,y))
            self.crop_ROI()

            # Draw rectangle around region of interst(ROI)
            cv2.rectangle(self.clone, self.image_coordinates[0], self.image_coordinates[1], (255,0,0), 2)
            cv2.imshow("image", self.clone) 

        # Clear all drawn boxes on right mouse button click
        elif event == cv2.EVENT_RBUTTONDOWN:
            self.clone = self.original_image.copy()

    def crop_ROI(self):
        self.cropped_image = self.clone.copy()
        self.cropped_image = self.cropped_image[self.image_coordinates[0][1]:self.image_coordinates[1][1],
                                                self.image_coordinates[0][0]:self.image_coordinates[1][0]]

        print('Cropped image: {} {}'.format(self.image_coordinates[0], self.image_coordinates[1]))
    
    def save_with_augmentation(self):
        # Generating 6 photos rotated 90 degrees
        # If you want you can use another augmentation techniques
        samples = expand_dims(self.cropped_image, 0)
        datagen = ImageDataGenerator(rotation_range=90)
        it = datagen.flow(samples, batch_size=1)
        
        for i in range(6):
            batch = it.next()
            image = batch[0].astype('uint8')
            cv2.imwrite('augmentation' + str(i) + str('.jpg'),image)
            
    def show_image(self):
        return self.clone
    
    def show_cropped_ROI(self):
        cv2.imshow('cropped image', self.cropped_image)
        
    def save_cropped_ROI(self):
        cv2.imwrite('crop.jpg',self.cropped_image)
        
if __name__ == '__main__':
    extract_image = ExtractImage('kediresmi.jpg') # Write your image name
    while True:
        cv2.imshow('image', extract_image.show_image())
        key = cv2.waitKey(1)

        # Close program with keyboard 'q'
        if key == ord('q'):
            cv2.destroyAllWindows()
            cv2.waitKey(0)

        # Crop image
        if key == ord('c'):
            extract_image.show_cropped_ROI()
            
        # Save cropped image
        if key == ord('s'):
            extract_image.save_cropped_ROI()
            print('Cropped image saved !')
            
        #Save cropped image with augmentation 
        if key == ord('a'):
            extract_image.save_with_augmentation()
            print('Augmented cropped images saved !')